#Tarefa 5 - Regressão Linear e logística

## Nesta tarefa, você deve carregar um dataset sobre as vendas de cadeirinhas infantis, construir modelo de Regressão com os algoritmos vistos em aula e predizer o valor das vendas.

Dica: Para toda a tarefa, além da biblioteca pandas e numpy, você pode querer explorar funções da biblioteca sklearn.linear_model (em particular os pacotes LinearRegression e LogisticRegression).
Além disso, você vai precisar usar funções de pré-processamento e de pós-procesamento (das bibliotecas sklearn.preprocessing, sklearn.model_selection e sklearn.metrics)


###Importe os pacotes e carregue o arquivo com os dados
O dataset a ser utilizado encontra-se no arquivo casseats.csv, disponível no EAD.


Este dataset contém dados sobre as vendas de cadeirinhas infantis em 400 lojas diferentes, descritas pelos seguintes atributos/variáveis:
* Sales - Vendas unitárias (em milhares) em cada local
* CompPrice - Preço cobrado pelo concorrente em cada local
* Income - Nível de renda da comunidade (em milhares de dólares)
* Advertising - Orçamento de publicidade local para a empresa em cada local (em milhares de dólares)
* Population - Tamanho da população na região (em milhares)
* Price - Preço que a empresa cobra por assentos de carro em cada local
* ShelveLoc - Um fator com níveis Ruim, Bom e Médio que indica a qualidade da localização das estantes para as cadeirinhas em cada local
* Age - Idade média da população local
* Education - Nível de escolaridade em cada localidade
* Urban - Um fator com níveis Não e Sim para indicar se a loja está localizada em área urbana ou rural
* US - Um fator com níveis Não e Sim para indicar se a loja é nos EUA ou não
* High - Um fator com níveis Não e Sim para indicar se a compra é alta ou não

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix, roc_curve, auc
import numpy as np

In [ ]:
# Carregar a base de dados
df = pd.read_csv('Carseats.csv')



### Pré-processe a base de dados

Dica: avalie a necessidade de converter os tipos das variáveis, normalizar os dados, ...

### Crie os conjuntos de treinamento e de teste

Atenção: Selecione aleatoriamente e sem reposição (para que não se repitam) 320 registros para o conjunto de treinamento. As 80 observações restantes serão usadas para o conjunto de teste. Fixe a semente de geração de dados aleatórios em 456.

In [ ]:
# Dividir os dados em conjunto de treino e teste


In [ ]:
# visualizar os dados


### Construa um modelo de Regressão Linear Múltipla.

Utilizando o modelo, faça a predição do atributo Sales no conjunto teste

Atenção: o atributo High deve ser eliminado para este processo

### Pós-processamento: Avalie cada modelo de regressão

Calcule as medidas de desempenho vistas em aula (erro quadrático médio, R2)

In [ ]:
# Calcular métricas de desempenho para cada modelo


### Agora, construa um modelo de Regressão Logística

Faça a predição do atributo High no conjunto teste, e avalie utilizando métricas adequadas

Atenção: o atributo Sales deve ser eliminado para este processo

In [ ]:
# Didivir dados de treino e teste

# Treinar o modelo de Regressão Logística

# Fazer predições com o conjunto de teste

# Calcular métricas de desempenho
